# 首先应当搭建好tcp通信基础

## 头文件

In [ ]:
#ifndef _TCP_H_
#define _TCP_H_
/*设置条件编译以防止重复编译*/

#include <stdio.h>
#include <sys/socket.h>
#include <netinet/in.h>
#include <netinet/tcp.h>
#include <stdlib.h>
#include <unistd.h>
#include <strings.h>
#include <arpa/inet.h>
#include <errno.h>

#define BACKLOG 5

#define ErrExit(msg) do {perror(msg); exit(EXIT_FAILURE); } while(0)

typedef struct sockaddr Addr;
typedef struct sockaddr_in Addr_in;

#endif

## 服务端

In [ ]:
#include "tcp.h"

int main(int argc, char *argv[])
{
	//int fd, newfd;
	//int ret;
	//char buf[BUFSIZ];
	//Addr_in addr, client_addr;
	//socklen_t addrlen = sizeof(addr);
	/*1. 检查参数*/
	if(argc < 3)
	{
		fprintf(stderr, "%s <addr><port>\n", argv[0]);
		exit(EXIT_FAILURE);
	}

	/*2. 创建套接字*/
	int fd;
	if((fd = socket(AF_INET, SOCK_STREAM, 0)) < 0)  //注册使用AF_INET协议族，SOCK_STREAM流式套接字为TCP协议
		ErrExit("socket");
	/*设置通信结构体*/
	Addr_in addr, client_addr;  //通信地址结构体
	bzero(&addr, sizeof(addr));
	addr.sin_family = AF_INET;  //使用AF_INET协议族
	addr.sin_port = htons(atoi(argv[2]));  //端口号
	if(inet_aton(argv[1], &addr.sin_addr) == 0) 
	{
		fprintf(stderr, "Invalid address\n");
		exit(EXIT_FAILURE);
	}
	/*绑定通信结构体*/
	if(bind(fd, (Addr *)&addr, sizeof(addr)))
		ErrExit("bind");

	/*监听模式*/
	if(listen(fd, BACKLOG))
		ErrExit("listen");

	/*接收客户端连接*/
	int newfd;
	socklen_t addrlen = sizeof(addr);
	do 
	{
		newfd = accept(fd, (Addr *)&client_addr, &addrlen);  //等待客户端连接
	}
	while(newfd < 0 && errno == EINTR); //如果信号导致的错误，继续执行
	if(newfd < 0)
		ErrExit("accept");

	/*接收客户端数据*/
	int ret;
	char buf[BUFSIZ];  //接收缓冲区
	while(1)
	{
		do 
		{
			ret = recv(newfd, buf, BUFSIZ, 0);  //接收数据
		}
		while(ret < 0 && errno == EINTR); //如果信号导致的错误，继续执行
		if(ret < 0)  //如果其他错误，退出
			ErrExit("recv");
		else if(!ret)  //如果客户端关闭，recv返回0
			break;
		else
			printf("[%s:%d]buf:%s\n", 
					inet_ntoa(client_addr.sin_addr), 
					ntohs(client_addr.sin_port), 
					buf);
	}
	close(newfd);
	close(fd);
	return 0;
}

## 客户端

In [ ]:
#include "tcp.h"

int main(int argc, char *argv[])
{
	//int fd;
	//int ret;
	//char buf[BUFSIZ] = {"===test==="};
	//Addr_in addr;
	/*检查参数*/
	if(argc < 3)
    {
		fprintf(stderr, "%s <addr><port>\n", argv[0]);
		exit(EXIT_FAILURE);
	}

	/*创建套接字*/
    int fd;
	if((fd = socket(AF_INET, SOCK_STREAM, 0)) < 0)
		ErrExit("socket");
	/*设置通信结构体*/
    Addr_in addr;
	bzero(&addr, sizeof(addr));
	addr.sin_family = AF_INET;
	addr.sin_port = htons(atoi(argv[2]));
	if(inet_aton(argv[1], &addr.sin_addr) == 0) 
    {
		fprintf(stderr, "Invalid address\n");
		exit(EXIT_FAILURE);
	}

	/*发起连接请求*/
	if(connect(fd, (Addr *)&addr, sizeof(addr)))
		ErrExit("connect");

	/*发送数据*/
	int ret;
    char buf[BUFSIZ] = {"===test==="};
    while(1)
    {
		do {
			ret = send(fd, buf, BUFSIZ, 0);
		}while(ret < 0 && errno == EINTR); //如果信号导致的错误，继续执行
		if(ret < 0)
			ErrExit("recv");
		else if(!ret)
			break;
		printf("send data:%s", buf);
		fflush(stdout);
		getchar();
	}

	close(fd);
	return 0;
}

## 二、实现函数封装

In [ ]:
#ifndef _TCP_H_
#define _TCP_H_

#include <stdio.h>
#include <sys/socket.h>
#include <netinet/in.h>
#include <netinet/tcp.h>
#include <stdlib.h>
#include <unistd.h>
#include <strings.h>
#include <arpa/inet.h>
#include <errno.h>

#define BACKLOG 5

#define ErrExit(msg) do { perror(msg); \
	exit(EXIT_FAILURE); } while(0)

typedef struct sockaddr Addr;
typedef struct sockaddr_in Addr_in;

void Argment(int argc, char *argv[]);
int SocketInit(char *argv[]);
#endif

## 函数封装

In [ ]:
#include "tcp.h"

void Argment(int argc, char *argv[]) //检查参数
{
	if(argc < 3)
	{
		fprintf(stderr, "%s <addr><port>\n", argv[0]);
		exit(EXIT_FAILURE);
	}
}

int SocketInit(char *argv[])   //创建套接字并发起连接请求
{
	//int fd;
	//Addr_in addr;
	/*创建套接字*/
	int fd;
	if((fd = socket(AF_INET, SOCK_STREAM, 0)) < 0)
		ErrExit("socket");
	/*设置通信结构体*/
	Addr_in addr;
	bzero(&addr, sizeof(addr));
	addr.sin_family = AF_INET;
	addr.sin_port = htons(atoi(argv[2]));
	if(inet_aton(argv[1], &addr.sin_addr) == 0)  
	{
		fprintf(stderr, "Invalid address\n");
		exit(EXIT_FAILURE);
	}
	/*发起连接请求*/
	if(connect(fd, (Addr *)&addr, sizeof(addr)))
		ErrExit("connect");
	return fd;
}

## 客户端

In [ ]:
#include "tcp.h"

int main(int argc, char *argv[])
{
	//int fd;
	//int ret;
	//char buf[BUFSIZ] = {"===test==="};
	/*检查参数*/
	Argment(argc, argv);
    /*创建套接字并发起连接请求*/
    int fd;
	fd = SocketInit(argv);
	/*发送数据*/
    int ret;
    char buf[BUFSIZ] = {"===test==="};
	while(1)
    {
		do
        {
			ret = send(fd, buf, BUFSIZ, 0);
		}
        while(ret < 0 && errno == EINTR); //如果信号导致的错误，继续执行
		
        if(ret < 0)
			ErrExit("recv");
		else if(!ret)
			break;
		printf("send data:%s", buf);
		fflush(stdout);
		getchar();
	}

	close(fd);
	return 0;
}

## 三、封装函数的再次升级

In [ ]:
#include "tcp.h"

void Argment(int argc, char *argv[])
{
	if(argc < 3)
    {
		fprintf(stderr, "%s <addr><port>\n", argv[0]);
		exit(EXIT_FAILURE);
	}
}

int SocketInit(char *argv[], bool server)
{
	int fd;
	Addr_in addr;
	func_t func = server?bind:connect;
	/*创建套接字*/
	if((fd = socket(AF_INET, SOCK_STREAM, 0)) < 0)
		ErrExit("socket");
	/*设置通信结构体*/
	bzero(&addr, sizeof(addr));
	addr.sin_family = AF_INET;
	addr.sin_port = htons(atoi(argv[2]));
	if (inet_aton(argv[1], &addr.sin_addr) == 0) 
    {
		fprintf(stderr, "Invalid address\n");
		exit(EXIT_FAILURE);
	}

	/*发起连接请求或绑定地址*/
	if(func(fd, (Addr *)&addr, sizeof(addr)))
		ErrExit("connect");

	if(server)
    {
		/*地址快速重用*/
		int b_reuse = 1;
		setsockopt(fd, SOL_SOCKET, SO_REUSEADDR, &b_reuse, sizeof(int));

		/*监听模式*/
		if(listen(fd, BACKLOG))
			ErrExit("listen");

	}
	return fd;
}

int SocketDataHandle(int fd, void *buf, size_t len, DataHand_t datahandle)  //数据处理
{
	int ret;
	char *str = datahandle == recv ? "recv" :"send";  //判断是接收还是发送
	do 
	{
		ret = datahandle(fd, buf, len, 0);
	} 
	while(ret < 0 && errno == EINTR);
	if(ret < 0)
		ErrExit(str);
	return ret;
}

In [ ]:
#include "tcp.h"

int main(int argc, char *argv[])
{
	int fd, newfd;
	int ret;
	char buf[BUFSIZ];
	Addr_in client_addr;
	socklen_t addrlen = sizeof(Addr_in);
	/*检查参数*/
	Argment(argc, argv);
	/*创建服务端套接字*/
	fd = SocketInit(argv, true);

	/*接收客户端连接*/
	do 
	{
		newfd = accept(fd, (Addr *)&client_addr, &addrlen);
	}
	while(newfd < 0 && errno == EINTR); //如果信号导致的错误，继续执行
	if(newfd < 0)
		ErrExit("accept");

	/*接收客户端数据*/
	while(1)
	{
		ret = SocketDataHandle(newfd, buf, BUFSIZ, (DataHand_t)recv);
		if(!ret)
			break;
		else
			printf("[%s:%d]buf:%s\n", 
					inet_ntoa(client_addr.sin_addr), 
					ntohs(client_addr.sin_port), buf);
	}

	close(newfd);
	close(fd);
	return 0;
}

In [ ]:
#include "tcp.h"

int main(int argc, char *argv[])
{
	int fd;
	int ret;
	char buf[BUFSIZ] = {"===test==="};
	/*检查参数*/
	Argment(argc, argv);

	fd = SocketInit(argv, false);

	/*发送数据*/
	while(1)
	{
		ret = SocketDataHandle(fd, buf, strlen(buf), (DataHand_t)send);
		if(!ret)
			break;
		printf("send data:%s", buf);
		fflush(stdout);
		getchar();
	}

	close(fd);
	return 0;
}

In [ ]:
#ifndef _TCP_H_
#define _TCP_H_

#include <stdio.h>
#include <sys/socket.h>
#include <netinet/in.h>
#include <netinet/tcp.h>
#include <stdlib.h>
#include <unistd.h>
#include <strings.h>
#include <arpa/inet.h>
#include <errno.h>
#include <stdbool.h>
#include <string.h>

#define BACKLOG 5

#define ErrExit(msg) do { perror(msg); exit(EXIT_FAILURE); } while(0)


typedef struct sockaddr Addr;
typedef struct sockaddr_in Addr_in;

typedef int (* func_t)(int, const Addr *, socklen_t); //函数指针类型
typedef ssize_t(* DataHand_t)(int, void *, size_t, int);  //数据处理函数指针类型 //ssize_t是有符号整型

void Argment(int argc, char *argv[]);  //检查参数
int SocketInit(char *argv[], bool server);  //	创建套接字并发起连接请求
int SocketDataHandle(int fd, void *buf, size_t len, DataHand_t datahandle);  //数据处理
#endif

# 接下来在此基础上实现文件传输

## 头文件

In [ ]:
#ifndef _TCP_H_
#define _TCP_H_

#include <stdio.h>
#include <sys/socket.h>
#include <netinet/in.h>
#include <netinet/tcp.h>
#include <stdlib.h>
#include <unistd.h>
#include <strings.h>
#include <arpa/inet.h>
#include <errno.h>
#include <stdbool.h>
#include <string.h>

#define BACKLOG 5  //监听队列长度

#define ErrExit(msg) do { perror(msg); exit(EXIT_FAILURE); } while(0)


typedef struct sockaddr Addr;
typedef struct sockaddr_in Addr_in;

typedef int (* func_t)(int, const Addr *, socklen_t);
typedef ssize_t(* DataHand_t)(int, void *, size_t, int);

void Argment(int argc, char *argv[]);
int SocketInit(char *argv[], bool server);
int SocketDataHandle(int fd, void *buf, size_t len, DataHand_t datahandle);

#include <sys/stat.h>
#include <fcntl.h>

#define OK '1'

#endif

## 函数定义

In [ ]:
#include "tcp.h"

void Argment(int argc, char *argv[])
{
	if(argc < 3)
	{
		fprintf(stderr, "%s <addr><port>\n", argv[0]);
		exit(EXIT_FAILURE);
	}
}

int SocketInit(char *argv[], bool server)
{
	int fd;
	Addr_in addr;
	func_t func = server?bind:connect;
	/*创建套接字*/
	if((fd = socket(AF_INET, SOCK_STREAM, 0)) < 0)
		ErrExit("socket");
	/*设置通信结构体*/
	bzero(&addr, sizeof(addr));
	addr.sin_family = AF_INET;
	addr.sin_port = htons( atoi(argv[2]));
	if (inet_aton(argv[1], &addr.sin_addr) == 0) 
	{
		fprintf(stderr, "Invalid address\n");
		exit(EXIT_FAILURE);
	}

	/*发起连接请求或绑定地址*/
	if(func(fd, (Addr *)&addr, sizeof(addr)))
		ErrExit("connect");

	if(server)
	{
		/*地址快速重用*/
		int b_reuse = 1;
		setsockopt(fd, SOL_SOCKET, SO_REUSEADDR, &b_reuse, sizeof(int));

		/*监听模式*/
		if(listen(fd, BACKLOG))
			ErrExit("listen");

	}
	return fd;
}

int SocketDataHandle(int fd, void *buf, size_t len, DataHand_t datahandle)
{
	int ret;
	char *str = datahandle == recv ? "recv" :"send";
	do 
	{
		ret = datahandle(fd, buf, len, 0);
	} 
	while(ret < 0 && errno == EINTR);
	if(ret < 0)
		ErrExit(str);
	return ret;
}

## 服务端

In [ ]:
#include "tcp.h"

int main(int argc, char *argv[])
{
	int fd, newfd, file_fd;
	int ret;
	char buf[BUFSIZ] = {};
	Addr_in client_addr;
	socklen_t addrlen = sizeof(Addr_in);
	/*检查参数*/
	Argment(argc, argv);
	/*创建服务端套接字*/
	fd = SocketInit(argv, true);

	/*接收客户端连接*/
	do 
	{
		newfd = accept(fd, (Addr *)&client_addr, &addrlen);
	}
	while(newfd < 0 && errno == EINTR); //如果信号导致的错误，继续执行
	if(newfd < 0)
		ErrExit("accept");

	/*接收文件名字*/
	ret = SocketDataHandle(newfd, buf, BUFSIZ, recv);
/********  文件处理部分  *********/
	/*创建文件*/
	if((file_fd = open(buf, O_WRONLY|O_CREAT, 0660)) < 0)  //以读写方式打开文件，如果文件不存在则创建
		ErrExit("file_fd");
	buf[0] = OK;
	SocketDataHandle(newfd, buf, 1, (DataHand_t)send);
	/*接收文件*/
	while(1)
	{
		ret = SocketDataHandle(newfd, buf, BUFSIZ, recv);
		if(!ret)
			break;
		write(file_fd, buf, ret);    //将接收到的数据写入文件
	}

	close(file_fd);
	close(newfd);
	close(fd);
	return 0;
}

In [ ]:
#include "tcp.h"
#define FILENAME "picture.png"

int main(int argc, char *argv[])
{
	int fd, file_fd;
	int ret;
	char buf[BUFSIZ];
	/*检查参数*/
	Argment(argc, argv);

	fd = SocketInit(argv, false);

	/*打开文件*/
	if((file_fd = open(FILENAME, O_RDONLY)) < 0)  //	O_RDONLY 只读
		ErrExit("open");

	/*发送文件名字*/
	SocketDataHandle(fd, FILENAME, strlen(FILENAME), (DataHand_t)send);
	SocketDataHandle(fd, buf, 1, recv);
	
	/*发送文件*/
	if(buf[0] == OK)
	{
		while(1)
		{
			do 
			{
				ret = read(file_fd, buf, BUFSIZ);
			}
			while(ret < 0 && errno == EINTR);
			
			if( ret < 0)
				ErrExit("read");
			if(!ret)
				break;
			ret = SocketDataHandle(fd, buf, ret, (DataHand_t)send);
			if(!ret)
				break;
		}
	}

	close(file_fd);
	close(fd);
	return 0;
}

# 实现文件传输

In [ ]:
#ifndef _TCP_H_
#define _TCP_H_

#include <stdio.h>
#include <sys/socket.h>
#include <netinet/in.h>
#include <netinet/tcp.h>
#include <stdlib.h>
#include <unistd.h>
#include <strings.h>
#include <arpa/inet.h>
#include <errno.h>
#include <stdbool.h>
#include <string.h>
#include <sys/stat.h>
#include <fcntl.h>

#define BACKLOG 5
#define OK '1'
#define ErrExit(msg) do { perror(msg); exit(EXIT_FAILURE); } while(0)


typedef struct sockaddr Addr;
typedef struct sockaddr_in Addr_in;

typedef int (* func_t)(int, const Addr *, socklen_t);
typedef ssize_t(* DataHand_t)(int, void *, size_t, int);

void Argment(int argc, char *argv[]);
int SocketInit(char *argv[], bool server);
int SocketDataHandle(int fd, void *buf, size_t len, DataHand_t datahandle);

#endif

In [ ]:
#include "tcp.h"

void Argment(int argc, char *argv[])
{
	if(argc < 3)
    {
		fprintf(stderr, "%s <addr><port>\n", argv[0]);
		exit(EXIT_FAILURE);
	}
}

int SocketInit(char *argv[], bool server)
{
	int fd;
	Addr_in addr;
	func_t func = server?bind:connect;
	/*创建套接字*/
	if( (fd = socket(AF_INET, SOCK_STREAM, 0) ) < 0)
		ErrExit("socket");
	/*设置通信结构体*/
	bzero(&addr, sizeof(addr) );
	addr.sin_family = AF_INET;
	addr.sin_port = htons( atoi(argv[2]) );
	if (inet_aton(argv[1], &addr.sin_addr) == 0) 
    {
		fprintf(stderr, "Invalid address\n");
		exit(EXIT_FAILURE);
	}

	/*发起连接请求或绑定地址*/
	if( func(fd, (Addr *)&addr, sizeof(addr) ) )
		ErrExit("connect");

	if(server)
    {
		/*地址快速重用*/
		int b_reuse = 1;
		setsockopt(fd, SOL_SOCKET, SO_REUSEADDR, &b_reuse, sizeof(int) );

		/*监听模式*/
		if( listen(fd, BACKLOG) )
			ErrExit("listen");
	}
	return fd;
}

int SocketDataHandle(int fd, void *buf, size_t len, DataHand_t datahandle)  //接收或者发送数据
{
	int ret;
	char *str = datahandle == recv ? "recv" :"send";
	do 
    {
		ret = datahandle(fd, buf, len, 0);
	} 
    while(ret < 0 && errno == EINTR);
	if(ret < 0)
		ErrExit(str);
	return ret;
}

In [ ]:
#include "tcp.h"

#define INFOFILE ".info"

int main(int argc, char *argv[])
{
	int fd, file_fd;
	int ret;
	FILE *fp;
	char buf[BUFSIZ];
	char *filename = argv[1];

	if(argc < 2)
	{
		printf("%s <filename>\n", argv[0]);
		exit(0);
	}

	/*通过配置文件获取IP地址和端口号*/
	if( (fp = fopen(INFOFILE, "r") ) == NULL) //打开配置文件
		ErrExit("fopen");
	fgets(buf, 20, fp);  //读取IP地址
	buf[strlen(buf)-1] = '\0'; //去掉换行符
	argv[1] = buf;  //将IP地址赋值给argv[1]
	fgets(&buf[20], 20, fp); //读取端口号
	buf[strlen(&buf[20])-1+20] = '\0';  //去掉换行符
	argv[2] = &buf[20];  //将端口号赋值给argv[2]

	fd = SocketInit(argv, false);  

	/*打开文件*/
	if( (file_fd = open(filename, O_RDONLY) ) < 0)
		ErrExit("open");

	/*发送文件名字*/
	SocketDataHandle(fd, filename, strlen(filename), (DataHand_t)send);  //发送文件名字
	SocketDataHandle(fd, buf, 1, recv);  //接收确认信息
	
	/*发送文件*/
	if(buf[0] == OK)
	{
		while(1)
		{
			do 
			{
				ret = read(file_fd, buf, BUFSIZ);  //读取文件
			}
			while(ret < 0 && errno == EINTR);
			if( ret < 0)
				ErrExit("read");
			if(!ret)
				break;
			ret = SocketDataHandle(fd, buf, ret, (DataHand_t)send);  //发送文件
			if(!ret)
				break;
			printf("ret = %d\n", ret);
		}
	}

	close(file_fd);
	close(fd);
	return 0;
}

In [ ]:
#include "tcp.h"

int main(int argc, char *argv[])
{
	int fd, newfd, file_fd;
	int ret;
	char buf[BUFSIZ] = {};
	Addr_in client_addr;
	socklen_t addrlen = sizeof(Addr_in);
	/*检查参数*/
	Argment(argc, argv);
	/*创建服务端套接字*/
	fd = SocketInit(argv, true);

	/*接收客户端连接*/
	do 
	{
		newfd = accept(fd, (Addr *)&client_addr, &addrlen);
	}
	while(newfd < 0 && errno == EINTR); //如果信号导致的错误，继续执行
	if(newfd < 0)
		ErrExit("accept");

	/*接收文件名字*/
	ret = SocketDataHandle(newfd, buf, BUFSIZ, recv);

	/*创建文件*/
	if( (file_fd = open(buf, O_WRONLY|O_CREAT, 0660) ) < 0)  //创建文件
		ErrExit("file_fd");
	buf[0] = OK;  //发送确认信息
	SocketDataHandle(newfd, buf, 1, (DataHand_t)send);  //发送确认信息
	/*接收文件*/
	while(1)
	{
		ret = SocketDataHandle(newfd, buf, BUFSIZ, recv);  //接收文件
		if(!ret)
			break;
		write(file_fd, buf, ret); //写入文件
	}

	close(file_fd);
	close(newfd);
	close(fd);
	return 0;
}

# 实现云同步

## TCP通信头文件

In [ ]:
#ifndef _NET_H_
#define _NET_H_

#include <stdio.h>
#include <sys/socket.h>
#include <netinet/in.h>
#include <netinet/tcp.h>
#include <stdlib.h>
#include <unistd.h>
#include <strings.h>
#include <arpa/inet.h>
#include <errno.h>
#include <stdbool.h>
#include <string.h>

#define BACKLOG 5

#define ErrExit(msg) do { \
	fprintf(stderr, "[%s:%d] ", __FUNCTION__, __LINE__); \ //打印出错的函数名和行号
	perror(msg); \  //打印错误信息
	exit(EXIT_FAILURE); } while(0)  //退出程序

typedef struct sockaddr Addr;  //通用地址结构体
typedef struct sockaddr_in Addr_in;  //IPv4地址结构体

void Argment(int argc, char *argv[]);  //检查参数
int SocketInit(char *addr, char *port, bool server);  //创建套接字并发起连接请求

ssize_t Read(int fd, void *buf, size_t len);  //读取数据
ssize_t Write(int fd, const void *buf, size_t len);  //发送数据

#endif

## TCP通信所需函数

In [ ]:
#include "net.h"

int SocketInit(char *str_ip, char *str_port, bool server)
{
	int fd;
	Addr_in addr;

	/*创建套接字*/
	if((fd = socket(AF_INET, SOCK_STREAM, 0)) < 0)
		ErrExit("socket");

	/*设置通信结构体*/
	bzero(&addr, sizeof(addr));
	addr.sin_family = AF_INET;
	addr.sin_port = htons(atoi(str_port));
	if (inet_aton(str_ip, &addr.sin_addr) == 0) 
	{
		fprintf(stderr, "Invalid address\n");
		exit(EXIT_FAILURE);
	}

	if(server)
	{ 	//判断是否为服务端
		/*地址快速重用*/
		int b_reuse = 1;
		setsockopt(fd, SOL_SOCKET, SO_REUSEADDR, &b_reuse, sizeof(int) );

		/*绑定地址*/
		if(bind(fd, (Addr *)&addr, sizeof(addr)))
			ErrExit("bind");

		/*设定为监听模式*/
		if(listen(fd, BACKLOG))
			ErrExit("listen");

	} 
	else  /*如果是客户端就发起连接请求*/
		if(connect(fd, (Addr *)&addr, sizeof(addr)))
			ErrExit("connect");

	return fd;
}

ssize_t Read(int fd, void *buf, size_t len) 
{
	ssize_t ret;  
	do 
	{
		ret = read(fd, buf, len);  //读取数据
	} 
	while(ret < 0 && errno == EINTR);
	if(ret < 0)
		ErrExit("read");
	return ret;
}

ssize_t Write(int fd, const void *buf, size_t len) 
{
	ssize_t ret;
	do 
	{
		ret = write(fd, buf, len);  //发送数据
	} 
	while(ret < 0 && errno == EINTR);
	if(ret < 0)
		ErrExit("write");
	return ret;
}


## 云同步所需头文件

In [ ]:
#ifndef _FILE_TRANSFER_H_
#define _FILE_TRANSFER_H_

#include "net.h"
#include <sys/stat.h>
#include <fcntl.h>
#include <dirent.h>
#include <time.h>

#define DEBUG printf("==debug information==\n"); //调试信息

#define CONFIG_FILE ".config"  //配置文件
#define SIZE_IP_STR 17  //IP地址字符串长度
#define SIZE_PORT_STR 6  //端口字符串长度
#define SIZE_PATH_STR 255  //路径字符串长度
#define SIZE_CLOUD_SPACE 1024*1024*1024   //云空间大小 单位:byte

#define SIZE_FT_TYPE 1  //自定义协议类型长度
#define SIZE_FT_F_SIZE 8  //文件大小长度

/*自定义协议类型*/
enum 
{
	TYPE_ERR, TYPE_OK, TYPE_GET, TYPE_PUT, TYPE_LIST, TYPE_SYNC,
};//错误，确认，获取文件，上传文件，获取文件列表，同步文件信息

/*自定义协议结构体*/
struct file_transfer 
{
	uint8_t type;
	/* GNU C的扩展属性，0长度的数组不占用内存空间
	 * msg 有效时，其他字段无效
	 * */
	char f_size[8];  //文件大小
	uint8_t len;  	 //文件名长度
	char f_name[0];  //文件名
	char msg[0];     //消息
	char f_body[0];  //文件内容
};

/*环境信息*/
struct config
{
	char ip[SIZE_IP_STR];   //IP地址
	char port[SIZE_PORT_STR];  //端口
};

/*文件链表*/
typedef struct node
{
	time_t mtime;  //文件修改时间
	char name[NAME_MAX];  //文件名
	struct node *next;  //下一个节点
}node_t;

/**客户端函数**/
/*********************************/
/*初始化环境*/
void FT_InitConfig(struct config *conf);
/*获取文件*/
void FT_GetFile(int sockfd, char *f_name, struct file_transfer *ft);
/*上传文件*/
void FT_PutFile(int sockfd, char *f_name, struct file_transfer *ft);
/*获取文件列表*/
void FT_FileList(int sockfd, struct file_transfer *ft);
/*同步文件信息*/
void FT_Sync(int sockfd, struct file_transfer *ft);
/*********************************/

/**服务端函数**/
/*********************************/
/*处理获取文件的请求*/
void FT_GetFileHandler(int sockfd, struct file_transfer *ft);
/*处理上传文件的请求*/
void FT_PutFileHandler(int sockfd, struct file_transfer *ft);
/*处理获取文件列表的请求*/
void FT_FileListHandler(int sockfd, struct file_transfer *ft);
/*同步文件信息*/
void FT_SyncHandler(int sockfd, struct file_transfer *ft);

#endif

## 云同步所需函数

In [ ]:
#include "file_transfer.h"

/*服务端私有函数*/
/*********************************/
/*创建文件,并写入空内容*/
static int W_zero(char *f_name, long long f_size) //f_name	文件名 f_size	文件大小
{
	int count, ret;
	char buf[BUFSIZ] = {};
	int f_fd = open(f_name, O_WRONLY|O_CREAT|O_TRUNC, 0640);  //创建文件
	if(f_fd < 0)  //如果创建文件失败
	{
		perror("open");
		return f_fd;
	}
	{
		perror("open");
		return f_fd;
	}
	/*占领磁盘空间(内核一次最多能写入BUFSIZ)*/
	while(f_size > 0)  //写入空内容
	{
		if(f_size > BUFSIZ)  //判断写入大小 
			count = BUFSIZ;  //一次最多写入BUFSIZ
		else 
			count = f_size;  //写入传入的文件大小
		ret = Write(f_fd, buf, count);  //将count大小的buf写入文件f_fd
		if(ret < 0)  //如果写入失败
			return ret;
		f_size = f_size - ret;  //如果写入成功 f_size剩余大小为实际大小减去实际写入大小
	}
	close(f_fd);  //关闭文件
	return 0;
}
/*接收文件*/
void W_body(int sockfd,char *f_name, long long f_size) //sockfd	文件描述符 f_name	文件名 f_size	文件大小
{
	char buf[BUFSIZ] = {};  //缓冲区用来存放接收的数据
	int f_fd, ret, count;  //f_fd	文件描述符 ret	接收数据大小 count	接收数据大小
	if((f_fd = open(f_name, O_WRONLY)) < 0)  //打开文件
		ErrExit("open");
	while(f_size > 0)  
	{
		if(f_size > BUFSIZ) 
			count = BUFSIZ;
		else 
			count = f_size;
		/*从网络文件描述符中获取数据写到本地文件描述符中*/
		ret = Read(sockfd, buf, count);  //从sockfd中读取count大小的数据到buf中
		Write(f_fd, buf, ret);  //将buf中的ret大小的数据写入f_fd
		f_size = f_size - ret;
	}
	close(f_fd);
}
/*发送文件*/
void R_body(int sockfd,char *f_name, long long f_size)
{
	char buf[BUFSIZ] = {};
	int f_fd, count, ret;
	/*打开文件*/
	if((f_fd = open(f_name, O_RDONLY)) < 0)
		return;
	while(f_size > 0)
	{
		if(f_size > BUFSIZ) 
			count = BUFSIZ;
		else 
			count = f_size;
		/*从本地文件描述符中获取数据写到网络文件描述符中*/
		ret = Read(f_fd, buf, count); //从f_fd中读取count大小的数据到buf中
		Write(sockfd, buf, ret);  //将buf中的ret大小的数据写入sockfd
		f_size -= ret;
	}
	close(f_fd);
}

/**客户端函数**/
/*********************************/
/*初始化环境*/
void FT_InitConfig(struct config *conf) 
{
	size_t len;
	/*从配置文件中获取IP地址和端口号*/
	FILE *fp = fopen(CONFIG_FILE, "r");  //打开配置文件
	if(fp == NULL)
		ErrExit("fopen");
	/*设置IP地址*/
	fgets(conf->ip, SIZE_IP_STR, fp);  //从fp中读取SIZE_IP_STR大小的数据到conf->ip中
	len = strlen(conf->ip);  //获取conf->ip的长度
	if(conf->ip[len-1] == '\n')  //如果conf->ip的最后一个字符是换行符
		conf->ip[len-1] = '\0';  //将conf->ip的最后一个字符设置为'\0'
	/*设置端口号*/
	fgets(conf->port, SIZE_IP_STR, fp);  //从fp中读取SIZE_IP_STR大小的数据到conf->port中
	len = strlen(conf->port);  //获取conf->port的长度
	if(conf->port[len-1] == '\n')  //如果conf->port的最后一个字符是换行符
		conf->port[len-1] = '\0';  //将conf->port的最后一个字符设置为'\0'
#ifdef DEBUG  
	printf("[%s:%d] conf->ip=%s\n", __FUNCTION__, __LINE__,
			conf->ip);
	printf("[%s:%d] conf->port=%s\n", __FUNCTION__, __LINE__, 
			conf->port);
#endif
	fclose(fp);
}
/*获取文件*/ /*客户端下载文件*/
void FT_GetFile(int sockfd, char *f_name, struct file_transfer *ft) 
//sockfd文件描述符 f_name文件名 ft文件传输结构体
{
	printf("[%s:%d]\n", __FUNCTION__, __LINE__);
	long long f_size;
	ft->type = TYPE_GET;  //消息类型为获取文件
	/*发送请求信息*/
	Write(sockfd, ft, SIZE_FT_TYPE);  //将ft中的SIZE_FT_TYPE大小的数据写入sockfd
	ft->len = strlen(f_name);  //获取f_name的长度
	Write(sockfd, &ft->len, 1);  //将ft->len大小的数据写入sockfd
	Write(sockfd, f_name, ft->len);  //将f_name中的ft->len大小的数据写入sockfd
	/*接收请求结果*/
	Read(sockfd, ft, SIZE_FT_TYPE);  //从sockfd中读取SIZE_FT_TYPE大小的数据到ft中
	if(ft->type == TYPE_OK)  //如果消息类型为确认
	{
		Read(sockfd, &f_size, SIZE_FT_F_SIZE);  //从sockfd中读取SIZE_FT_F_SIZE大小的数据到f_size中
		/*创建文件,并写入空内容*/
		W_zero(f_name, f_size);  
		/*写入文件内容*/
		W_body(sockfd, f_name, f_size);
	}
	else
	{
		/*如果接收文件失败，打印错误信息*/
		Read(sockfd, &ft->len, 1);
		Read(sockfd, ft->msg, ft->len);
		ft->msg[ft->len] = '\0';
		printf("get file [%s] failed [%s]\n", f_name, ft->msg);
	}
}
/*上传文件*/
void FT_PutFile(int sockfd, char *f_name, struct file_transfer *ft)
{
	printf("[%s:%d]\n", __FUNCTION__, __LINE__);
	/*打开文件*/
	int f_fd = open(f_name, O_RDONLY);
	if(f_fd < 0)
		ErrExit("open");
	/*获取文件属性*/
	struct stat st;
	if(fstat(f_fd, &st)) 
		ErrExit("stat");
	/*检查文件类型:只能上传普通文件*/
	if(!(st.st_mode & S_IFREG))  //如果不是普通文件
		return;
	/*获取文件大小*/
	long long f_size = (long long)st.st_size;
	/*设置自定义协议*/
	ft->type = TYPE_PUT; //消息类型
	Write(sockfd, ft, SIZE_FT_TYPE); //发送消息类型
	Write(sockfd, &f_size, SIZE_FT_F_SIZE); //发送文件大小
	ft->len = strlen(f_name);
	Write(sockfd, &ft->len, 1);
	Write(sockfd, f_name, ft->len);  //发送文件名字
	/*等待确认*/
	if( !Read(sockfd, ft, SIZE_FT_TYPE) )
		return;
	if(ft->type == TYPE_OK)
	{  //发送文件
		R_body(sockfd, f_name, f_size);
	}
	else
	{                    //如果确认失败，打印错误信息
		Read(sockfd, &ft->len, 1);
		Read(sockfd, ft->msg, ft->len);
		ft->msg[ft->len] = '\0';
		printf("get file [%s] failed [%s]\n", f_name, ft->msg);
	}
	/*关闭文件*/
	close(f_fd);
}
/*获取文件列表*/
void FT_FileList(int sockfd, struct file_transfer *ft)
{
	printf("[%s:%d]\n", __FUNCTION__, __LINE__);
	time_t mtime;
	ft->type = TYPE_LIST;
	Write(sockfd, &ft->type, SIZE_FT_TYPE);
	if( Read(sockfd, ft, SIZE_FT_TYPE) == 0)
		return;
	if(ft->type == TYPE_OK)
	{
		while(1)
		{
			Read(sockfd, &ft->len, 1);
			Read(sockfd, ft->f_name, ft->len);
			if(ft->len == 0)
				break;
			ft->f_name[ft->len] = '\0';
			printf("%-32s", ft->f_name);
			Read(sockfd, &mtime, sizeof(time_t) );
			printf("%s", ctime(&mtime) );
		}
	}
	else
	{
		printf("get list failed [%s]", ft->msg);
	}
}
/*同步文件信息*/
void FT_Sync(int sockfd, struct file_transfer *ft)
{
	printf("[%s:%d]\n", __FUNCTION__, __LINE__);
	DIR *dir;
	struct dirent *p;
	/*打开目录*/
	if( (dir = opendir(".") ) == NULL)
		ErrExit("opendir");
	/*读取目录*/
	while( (p = readdir(dir) ) != NULL )
	{
		if(p->d_type == DT_REG && p->d_name[0] != '.')
		{
			printf("输入任意键，继续上传文件%s\n", p->d_name);
			FT_PutFile(sockfd, p->d_name, ft);
		}
	}
	closedir(dir);
}
/*********************************/

/**服务端函数**/
/*********************************/
/*处理获取文件的请求*/
void FT_GetFileHandler(int sockfd, struct file_transfer *ft)
{
	printf("[%s:%d]\n", __FUNCTION__, __LINE__);
	Read(sockfd, &ft->len, 1); 
	Read(sockfd, ft->f_name, ft->len); 
	ft->f_name[ft->len] = '\0';
	/*获取文件属性*/
	struct stat st;
	if( stat(ft->f_name, &st) )
		ErrExit("stat");
	/*检查文件类型:只能下载普通文件*/
	char *errmsg = "只可以下载普通文件\n";
	if(!(st.st_mode & S_IFREG) || ft->f_name[0] == '.')
	{
		ft->type = TYPE_ERR;
		Write(sockfd, &ft->type, SIZE_FT_TYPE);
		ft->len = strlen(errmsg);
		Write(sockfd, &ft->len, 1);
		Write(sockfd, errmsg, ft->len);
		return;
	}
	ft->type = TYPE_OK;
	Write(sockfd, &ft->type, SIZE_FT_TYPE);
	long long f_size = (long long)st.st_size;
	Write(sockfd, &f_size, SIZE_FT_F_SIZE);
	/*发送文件内容*/
	R_body(sockfd, ft->f_name, f_size);
}
/*处理上传文件的请求*/
void FT_PutFileHandler(int sockfd, struct file_transfer *ft)
{
	printf("[%s:%d]\n", __FUNCTION__, __LINE__);
	long long f_size;
	/*创建文件*/
	Read(sockfd, &f_size, SIZE_FT_F_SIZE);
	Read(sockfd, &ft->len, 1);
	Read(sockfd, ft->f_name, ft->len);
	ft->f_name[ft->len] = '\0';
	if(ft->f_name[0] == '.')
	{
		ft->type = TYPE_ERR;
		sprintf(ft->msg, "不可以上传隐藏文件\n");
		Write(sockfd, ft, SIZE_FT_TYPE+strlen(ft->msg)+1);
		return;
	}
	int f_fd = open(ft->f_name, O_WRONLY|O_CREAT|O_TRUNC, 0640);
	if(f_fd < 0) 
	{
		ft->type = TYPE_ERR;
		Write(sockfd, &ft->type, SIZE_FT_TYPE);
		sprintf(ft->msg, "[创建打开失败][open:%s]\n", strerror(errno) );
		ft->len = strlen(ft->msg);
		Write(sockfd, &ft->len, 1);
		Write(sockfd, ft->msg, ft->len);
		printf("[send msg]%s\n", ft->msg);
		return;
	}
	/*占领磁盘空间(内核一次最多能写入BUFSIZ)*/
	if (W_zero(ft->f_name, f_size) < 0)
	{
		ft->type = TYPE_ERR;
		Write(sockfd, &ft->type, SIZE_FT_TYPE);
		sprintf(ft->msg, "[磁盘空间不足][write:%s]\n", strerror(errno) );
		ft->len = strlen(ft->msg);
		Write(sockfd, &ft->len, 1);
		Write(sockfd, ft->msg, ft->len);
		printf("[send msg]%s\n", ft->msg);
		return;
	}

	/*发送确认信息*/
	ft->type = TYPE_OK;
	if( Write(sockfd, ft, 1) < 0)
		return;
	/*写入文件内容*/
	W_body(sockfd, ft->f_name, f_size);
	printf("[文件接收成功]文件名: %-32s文件大小：%lld\n", ft->f_name, f_size);
}
/*处理获取文件列表的请求*/
void FT_FileListHandler(int sockfd, struct file_transfer *ft)
{
	printf("[%s:%d]\n", __FUNCTION__, __LINE__);
	DIR *dir;
	struct dirent *p;
	struct stat st;
	/*打开目录*/
	if( (dir = opendir(".") ) == NULL){
		ft->type = TYPE_ERR;
		sprintf(ft->msg, "[目录打开失败][opendir:%s]\n", strerror(errno) );
		Write(sockfd, ft, SIZE_FT_TYPE+strlen(ft->msg)+1);
		return;
	}
	/*读取目录*/
	ft->type = TYPE_OK;
	Write(sockfd, ft, SIZE_FT_TYPE);
	while( (p = readdir(dir) ) != NULL ){
		if(p->d_type == DT_REG && p->d_name[0] != '.'){
			ft->len = strlen(p->d_name);
			Write(sockfd, &ft->len, 1);
			Write(sockfd, p->d_name, ft->len);
			stat(p->d_name, &st);
			Write(sockfd, &st.st_mtime, sizeof(time_t) );
		}
	}
	ft->len = 0;
	Write(sockfd, &ft->len, 1);
	closedir(dir);
}
/*同步文件信息*/
void FT_SyncHandler(int sockfd, struct file_transfer *ft)
{
	printf("[%s:%d]\n", __FUNCTION__, __LINE__);
	DIR *dir;
	struct dirent *p;
	/*打开目录*/
	if( (dir = opendir(".") ) == NULL){
		ft->type = TYPE_ERR;
		sprintf(ft->msg, "[目录打开失败][opendir:%s]\n", strerror(errno) );
		Write(sockfd, ft, SIZE_FT_TYPE+strlen(ft->msg)+1);
		return;
	}
	/*读取目录*/
	while( (p = readdir(dir) ) != NULL ){
		if(p->d_type == DT_REG && p->d_name[0] != '.'){
			FT_GetFile(sockfd, p->d_name, ft);
		}
	}
	closedir(dir);
}


In [ ]:
#include "file_transfer.h"

void Client_Handle(int newfd, struct file_transfer *ft); //客户端处理函数

int main(int argc, char *argv[])
{
	char buf[BUFSIZ] = {};
	int fd, newfd;
	struct config conf;
	struct file_transfer *ft = (struct file_transfer *)buf;  //直接使用buf的内存空间

	/*根据config初始化环境*/
	FT_InitConfig(&conf);

	/*准备套接字*/
	fd = SocketInit(conf.ip, conf.port, true);

	/*循环处理客户端请求*/
	while(1)
	{
		/*接收来自客户端的连接*/
		newfd = accept(fd, NULL, NULL);  
		Client_Handle(newfd, ft);  //处理客户端请求
		close(newfd);
	}

	close(fd);

	return 0;
}

void Client_Handle(int newfd, struct file_transfer *ft)
{
	char *err_str = "Invalid option!";
	/*接收客户端数据*/
	while(Read(newfd, ft, SIZE_FT_TYPE) > 0)
	{
		switch(ft->type){
		case TYPE_GET://处理获取文件的请求
			FT_GetFileHandler(newfd, ft);
			break;
		case TYPE_PUT://处理上传文件的请求
			FT_PutFileHandler(newfd, ft);
			break;
		case TYPE_LIST://处理获取文件列表的请求
			FT_FileListHandler(newfd, ft);
			break;
		case TYPE_SYNC://同步文件信息
			FT_SyncHandler(newfd, ft);
			break;
		default:
			Write(newfd, err_str, strlen(err_str));
		}
	}
}

In [ ]:
#include "file_transfer.h"

void Usage(int argc, char *argv[]);

int main(int argc, char *argv[])
{
	char buf[BUFSIZ] = {};
	struct config conf;
	struct file_transfer *ft = (struct file_transfer *)buf;

	/*参数检查*/
	Usage(argc, argv);

	/*根据config初始化环境*/
	FT_InitConfig(&conf);

	/*准备套接字*/
	int fd = SocketInit(conf.ip, conf.port, false);

	/*执行相应功能*/
	switch(argv[1][1]) {
	case 'g': //获取文件
		FT_GetFile(fd, argv[2], ft);
		break;
	case 'p': //上传文件
		FT_PutFile(fd, argv[2], ft);
		break;
	case 'l'://获取文件列表
		FT_FileList(fd, ft);
		break;
	case 's'://同步文件信息
		FT_Sync(fd, ft);
		break;
	default:
		fprintf(stderr, "Invalid option!\n");
	}
	/*关闭文件描述符*/
	close(fd);

	return 0;
}

void Usage(int argc, char *argv[]){
	if(argc < 2){
		fprintf(stderr, "%s[type]<argment>\n", argv[0]);
		fprintf(stderr, "\t[type]<filename> -g: get file.\n");
		fprintf(stderr, "\t[type]<filename> -p: put file.\n");
		fprintf(stderr, "\t[type] -l: upload file list\n");
		fprintf(stderr, "\t[type] -s: File synchronization.\n");
		exit(EXIT_FAILURE);
	}
	if((argv[1][1] == TYPE_GET || argv[1][1] == TYPE_PUT) && argc < 3){
		fprintf(stderr, "\t[type]<filename> -g: get file.\n");
		fprintf(stderr, "\t[type]<filename> -p: put file.\n");
		exit(EXIT_FAILURE);
	}
}